In [67]:
import os
import torch
from transformers import AutoTokenizer,AutoModel, RobertaTokenizer

from torch import nn
class MultiClassClassifier(nn.Module):
    def __init__(self, bert_model_path, labels_count, hidden_dim=768, mlp_dim=500, extras_dim=100, dropout=0.1, freeze_bert=False):
        super().__init__()

        self.roberta = AutoModel.from_pretrained(bert_model_path,output_hidden_states=True,output_attentions=True)
        self.dropout = nn.Dropout(dropout)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, mlp_dim),
            nn.ReLU(),
            # nn.Linear(mlp_dim, mlp_dim),
            # # nn.ReLU(),
            # # nn.Linear(mlp_dim, mlp_dim),
            # nn.ReLU(),
            nn.Linear(mlp_dim, labels_count)
        )
        # self.softmax = nn.LogSoftmax(dim=1)
        if freeze_bert:
            print("Freezing layers")
            for param in self.roberta.parameters():
                param.requires_grad = False

    def forward(self, tokens, masks):
        output = self.roberta(tokens, attention_mask=masks)
        dropout_output = self.dropout(output["pooler_output"])
        # concat_output = torch.cat((dropout_output, topic_emb), dim=1)
        # concat_output = self.dropout(concat_output)
        mlp_output = self.mlp(dropout_output)
        # proba = self.sigmoid(mlp_output)
        # proba = self.softmax(mlp_output)

        return mlp_output
    



In [68]:

output_dir = "math_roberta_claimdecomp_final_continued"

# --- Parámetros usados durante el entrenamiento ---
bert_model_path = "uf-aice-lab/math-roberta"
labels_count = 3
hidden_dim = 1024
mlp_dim = 768
extras_dim = 140  # No se usa, pero estaba en tu init original
dropout = 0.1
freeze_bert = False

# --- Cargar tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(output_dir)

# --- Reinstanciar el modelo con la arquitectura original ---
model = MultiClassClassifier(
    bert_model_path,
    labels_count,
    hidden_dim=hidden_dim,
    mlp_dim=mlp_dim,
    extras_dim=extras_dim,
    dropout=dropout,
    freeze_bert=freeze_bert
)

# Mover el modelo a GPU
model.to('cuda')

# --- Cargar pesos guardados ---
model.load_state_dict(torch.load(os.path.join(output_dir, 'model_weights'), map_location=torch.device('cuda')))
model.eval()

# --- Función de inferencia ---
def classify_claim_and_evidence(claim, evidence):
    # Concatenar claim con evidencias (según tu formato)
    text = f"[Claim]: {claim} [Evidences]: {' '.join(evidence)}"
    
    # Tokenización
    inputs = tokenizer(
        text,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=512
    )

    # Pasar por el modelo
    with torch.no_grad():
        logits = model(inputs['input_ids'].to('cuda'), inputs['attention_mask'].to('cuda'))

    # Predicción
    predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class



Some weights of RobertaModel were not initialized from the model checkpoint at uf-aice-lab/math-roberta and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_4076630/1865101125.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_g

In [69]:
clases = ['Conflicting', 'False', 'True']

In [70]:


claim = "The company's software is user-friendly."
evidence = [
    "Users report that the interface is intuitive.",
    "Tutorials and support are readily available."
]

predicted_class = classify_claim_and_evidence(claim, evidence)
print(f"Predicted class: {clases[predicted_class]}")



Predicted class: False


In [71]:

## uno usado en el train, con label true
claim = "La subida del SMI a 900 euros es del 22%, la mayor desde 1977"
evidence = [
    " | 2 min lectura El Congreso de los Diputados ha celebrado la comparecencia en el Pleno del presidente del Gobierno, Pedro Sánchez, sobre Cataluña y el Brexit. Nada más comenzar su intervención, Sánchez hizo un anuncio: la subida del Salario Mínimo Interprofesional (SMI) que había pactado con Podemos en los Presupuestos Generales del Estado se aprobará en el próximo Consejo de Ministros del 21 de diciembre en Barcelona. Pero además, el presidente sacó pecho de lo que representa esta medida: «La subida del SMI a 900 euros es del 22%, la mayor desde 1977». Hemos comprobado los datos y la afirmación del presidente es VERDADERA. El SMI para 2018 quedó fijado en 735,9 euros, por lo que elevarlo a los 900 supondrá un crecimiento del 22,3%. Es la mayor subida desde 1977, cuando se fijó un SMI para el siguiente año, 1978, de 98,81 euros, un 24,5% mayor que los 79,33 de ese momento. Hasta que se apruebe la subida anunciada por el Gobierno, el mayor incremento en ese periodo es el que se produjo en 1983, del 13,1%. También hemos querido comprobar si esta subida es la mayor desde 1977 teniendo en cuenta el IPC; esto es, calculando esa subida con respecto a la subida del coste de la vida (tasa interanual de diciembre a diciembre). Si suben los precios, la capacidad adquisitiva disminuye pese a que suban los ingresos. En variación con respecto al IPC, la mayor subida fue la de 2017, de un 8% con un IPC del 1,1%; esto es, una variación de 6,9 puntos. Teniendo en cuenta que la subida para 2019 será del 22% y que Gobierno prevé que el deflactor del PIB, que mide la inflación, sea del 1,8%, esta subida será también la mayor teniendo en cuenta el IPC. Fuentes: Ministerio de Trabajo, Migraciones y Seguridad Social"
]

predicted_class = classify_claim_and_evidence(claim, evidence)
print(f"Predicted class: {clases[predicted_class]}")



Predicted class: False


In [72]:
# cargar validador
import json #English/test_set_english_claim.json
from sklearn.metrics import classification_report, confusion_matrix
with open("../../data/Spanish/val_spanish_claims.json", encoding='utf-8') as f:
  val_data = json.load(f)


true_labels = []
predicted_labels = []

for entry in val_data:
    claim = entry['claim']
    evidence = entry['doc']  # se espera que sea lista de strings
    label = entry['label']  # ← Asegúrate de que la clave correcta sea 'label'

    pred = classify_claim_and_evidence(claim, evidence)

    true_labels.append(label)
    predicted_labels.append(clases[pred])

# Reporte de métricas
print("Classification Report:\n")
print(classification_report(true_labels, predicted_labels, digits=4))

# Matriz de confusión
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))

Classification Report:

              precision    recall  f1-score   support

 Conflicting     0.0000    0.0000    0.0000        48
       False     0.7931    1.0000    0.8846       299
        True     0.0000    0.0000    0.0000        30

    accuracy                         0.7931       377
   macro avg     0.2644    0.3333    0.2949       377
weighted avg     0.6290    0.7931    0.7016       377


Confusion Matrix:
[[  0  48   0]
 [  0 299   0]
 [  0  30   0]]


/home/rumor/anaconda3/envs/clef/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rumor/anaconda3/envs/clef/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rumor/anaconda3/envs/clef/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

In [77]:
true_labels

['Conflicting',
 'False',
 'False',
 'False',
 'Conflicting',
 'Conflicting',
 'False',
 'Conflicting',
 'Conflicting',
 'Conflicting',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'True',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'Conflicting',
 'False',
 'False',
 'False',
 'False',
 'True',
 'False',
 'Conflicting',
 'True',
 'False',
 'False',
 'False',
 'False',
 'Conflicting',
 'False',
 'False',
 'False',
 'False',
 'False',
 'Conflicting',
 'False',
 'False',
 'True',
 'False',
 'Conflicting',
 'Conflicting',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'Conflicting',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'True',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'False',
 'Conflicting',
 'False',
 'False',
 